# Hedditch Authentication and Data Loading

In [ ]:
import requests
import pandas as pd

In [ ]:
# personal use script, secret token
auth = requests.auth.HTTPBasicAuth( 'JNpGyLI-oJ_drb93Dykw9g', 'KgImJaXURcpJG4Ud9-HLBsj8UnnnRQ' )

data = {'grant_type': 'password',
        'username': 'shawwquille_aww_neal',
        'password': 'rampat11' }

headers = {'User-Agent': 'SquatBotTUM/0.0.1' }

res = requests.post( 'https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers )

token1 = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {token1}"}}

requests.get( 'https://oauth.reddit.com/api/v1/me', headers=headers )

<Response [200]>

In [ ]:
res = requests.get( "https://oauth.reddit.com/r/stopdrinking/new", headers=headers, params={'limit': '100000'} )

print( res.json() )

{'kind': 'Listing', 'data': {'after': 't3_uw7w3p', 'dist': 100, 'modhash': None, 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'stopdrinking', 'selftext': "I'm really sad - I drank last night. I went on a first date with a guy and I think blew it. This isn't the first time I've done this either. :(", 'author_fullname': 't2_6pkycsnd', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Drank last night', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/stopdrinking', 'hidden': False, 'pwls': 0, 'link_flair_css_class': None, 'downs': 0, 'top_awarded_type': None, 'hide_score': True, 'name': 't3_uwnt26', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 1.0, 'author_flair_background_color': None, 'subreddit_type': 'public', 'ups': 3, 'total_awards_received': 0, 'media_embed': {}, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': [], 'secure_media': None, '

In [ ]:
df = pd.DataFrame()

for post in res.json()['data']['children']:
  df = df.append({
      'subreddit': post['data']['subreddit'],
      'title': post['data']['title'],
      'selftext': post['data']['selftext'],
      'upvote_ratio': post['data']['upvote_ratio'],
      'ups': post['data']['ups'],
      'downs': post['data']['downs'],
      'score': post['data']['score']
  }, ignore_index=True )

In [ ]:
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,stopdrinking,Drank last night,I'm really sad - I drank last night. I went on...,1.0,3.0,0.0,3.0
1,stopdrinking,30days,I am not sure how but today is 30 days alcohol...,1.0,3.0,0.0,3.0
2,stopdrinking,Stomach pain in the night?,I noticed since I stopped drinking I have cons...,1.0,1.0,0.0,1.0
3,stopdrinking,"Tuesday Morning, EST 6:19 am / IWNDWYT",Real B.S. meetings for work coming up today. ...,1.0,2.0,0.0,2.0
4,stopdrinking,How do you utilize SD? I’m curious how to best...,I’ve figured out my own way since I found y’al...,1.0,9.0,0.0,9.0


In [ ]:
raw_text1 = res.json()['data']['children'][4]['data']['selftext']

print( raw_text1 )

I’ve figured out my own way since I found y’all, and the thought was brought back up with a few recent posts/comments from newer members. *I want to get the most I can from SD and hopefully help others along the way.*

**Here’s what I do.**         
I come on in the AM, usually go straight to the DCI to check in. I *always* sort by new, because the folks who post earlier can get more “attention.” I go through &amp; upvote everyone’s IWNDWYT, depending on time. (Come back throughout the day to do several times.) *Some days I only have time to check in, I do what I can.*

I respond to people on the DCI randomly. Folks struggling, people I don’t see often, comments that resonate, or just say hi.  When someone replies to me, I upvote them. (I upvote a lot lol.)

**I don’t worry about upvotes. Ever.** I’d cry if I did 🤣. I purposefully don’t get notifications until I open the app &amp; click on the icon. If I don’t have engagement, I just engage more. I sort of treat it like a journal. 

**

# Sentence-level Clustering and Analysis

## Analyzing Titles

### spaCy and DBSCAN

In [ ]:
import spacy
from spacy import displacy
import spacy.cli

spacy.cli.download("en_core_web_lg")

NER = spacy.load( "en_core_web_lg" )

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
from tqdm import tqdm
from sklearn.cluster import DBSCAN
import numpy as np

In [ ]:
NLP = spacy.load( 'en_core_web_lg' )

sent_vecs = {}
docs = []

for title in tqdm( df.title ):
  doc = NLP( title )
  docs.append( doc )
  sent_vecs.update( {title: doc.vector} )

sentences = list( sent_vecs.keys() )
vectors = list( sent_vecs.values() ) 

100%|██████████| 100/100 [00:04<00:00, 24.76it/s]


In [ ]:
x = np.array( vectors )

n_classes = {}

for i in tqdm( np.arange( 0.001, 1, 0.002) ):
  dbscan = DBSCAN( eps=i, min_samples=2, metric='cosine' ).fit(x)
  n_classes.update( {i: len( pd.Series( dbscan.labels_ ).value_counts() ) } )

dbscan = DBSCAN( eps=0.099, min_samples=2, metric='cosine' ).fit(x) 

100%|██████████| 500/500 [00:01<00:00, 295.29it/s]


In [ ]:
n_classes

{0.001: 1,
 0.003: 1,
 0.005: 1,
 0.007: 1,
 0.009000000000000001: 1,
 0.011: 1,
 0.013000000000000001: 1,
 0.015: 1,
 0.017: 1,
 0.019000000000000003: 2,
 0.021: 2,
 0.023: 2,
 0.025: 2,
 0.027000000000000003: 2,
 0.029: 2,
 0.031: 2,
 0.033: 2,
 0.035: 2,
 0.037000000000000005: 2,
 0.039: 3,
 0.041: 3,
 0.043000000000000003: 3,
 0.045: 3,
 0.047: 3,
 0.049: 3,
 0.051000000000000004: 3,
 0.053000000000000005: 3,
 0.055: 3,
 0.057: 3,
 0.059000000000000004: 3,
 0.061: 3,
 0.063: 3,
 0.065: 3,
 0.067: 3,
 0.069: 3,
 0.07100000000000001: 4,
 0.07300000000000001: 4,
 0.075: 4,
 0.077: 4,
 0.079: 4,
 0.081: 4,
 0.083: 4,
 0.085: 4,
 0.08700000000000001: 4,
 0.089: 4,
 0.091: 3,
 0.093: 3,
 0.095: 3,
 0.097: 3,
 0.099: 3,
 0.101: 3,
 0.10300000000000001: 3,
 0.10500000000000001: 3,
 0.107: 3,
 0.109: 3,
 0.111: 4,
 0.113: 4,
 0.115: 5,
 0.117: 5,
 0.11900000000000001: 5,
 0.121: 4,
 0.123: 5,
 0.125: 5,
 0.127: 5,
 0.129: 5,
 0.131: 5,
 0.133: 4,
 0.135: 4,
 0.137: 4,
 0.139: 4,
 0.14100000

In [ ]:
print( dbscan.labels_ )

[-1 -1 -1 -1  0 -1 -1  0 -1 -1  0 -1  0 -1  1 -1 -1  0 -1 -1  0  0  0  0
  0 -1 -1 -1 -1 -1  0  0  0  0 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1  0 -1
 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0  0  0  1
 -1  0 -1  0 -1 -1 -1 -1  1 -1  0 -1 -1 -1 -1 -1 -1  0  0 -1 -1 -1  0 -1
  0  0 -1 -1]


In [ ]:
print( res.json()['data']['children'][17]['data']['title'] )

I would have laughed at this comment not too long ago… but it may help somebody


In [ ]:
for cluster in range( max( dbscan.labels_ ) + 1 ):
  print( "CLUSTER " + str( cluster ) + " SENTENCES:" )
  for lbl in range( len( dbscan.labels_ ) ):
    if dbscan.labels_[lbl] == cluster:
      print( res.json()['data']['children'][lbl]['data']['title'] )
  print( "" )

CLUSTER 0 SENTENCES:
How do you utilize SD? I’m curious how to best utilize this sub.
On Day 4 now went to a friend yesterday to help renovate his house.
I'm tapering at home with the guidance of a professional psychologist. 10+ years drinking is enough.
Day 3. I want to live my life.
I would have laughed at this comment not too long ago… but it may help somebody
I want to quit drinking and vaping, but I'm not strong enough to do both at once, and if I quit one, I get more heavy into the other.
187 Days and I'm Going to a Party This Weekend
So long as I follow that way I have nothing to fear. Just for today.
Lost my gf due to drinking every night. Found a supplement that really really took away my cravings.
Day 11 and starting to worry that the shaky, anxious, nervous thing is never going away.
Told my friend I'm sick
I sent this to myself just now
Who Did You Ask For Help?
Telling my friends tomorrow that I'm an alcoholic
I told my partner
Reasons we think we do vs reasons we actually

### NLTK with Doc2Vec

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def cosine(u, v):
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokenized_sentences = []

for s in tqdm( df.title ):
  tokenized_sentences.append( word_tokenize( s.lower() ) )

tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate( df.title )]

model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)

model.docvecs.vectors_docs

100%|██████████| 100/100 [00:00<00:00, 3853.50it/s]


array([[ 0.05463709,  0.04428192,  0.05420855, ...,  0.11807451,
         0.188307  ,  0.06715076],
       [-0.03833764, -0.08877381,  0.24552521, ...,  0.44495007,
         0.3461108 ,  0.09535689],
       [ 0.15647037,  0.20015702, -0.07400504, ...,  0.03189921,
         0.22788674, -0.03733876],
       ...,
       [ 0.14620216,  0.4449527 , -0.38120794, ..., -0.04940188,
         0.0600504 ,  0.34460476],
       [ 0.10112389,  0.12448607, -0.00523894, ...,  0.00099499,
         0.1939772 ,  0.02378425],
       [-0.05421987, -0.06833632,  0.18083003, ...,  0.41962036,
         0.31071225,  0.07858857]], dtype=float32)

In [ ]:
sent_vecs = {}

for i in range( model.docvecs.count ):
  sent_vecs.update( {df.title[i]: model.docvecs.vectors_docs[i]} )

sentences = list( sent_vecs.keys() )
vectors = list( sent_vecs.values() )

In [ ]:
x = np.array( vectors )

n_classes = {}

for i in tqdm( np.arange( 0.001, 1, 0.002) ):
  dbscan = DBSCAN( eps=i, min_samples=2, metric='cosine' ).fit(x)
  n_classes.update( {i: len( pd.Series( dbscan.labels_ ).value_counts() ) } )

dbscan = DBSCAN( eps=0.06, min_samples=2, metric='cosine' ).fit(x) 

100%|██████████| 500/500 [00:01<00:00, 262.21it/s]


In [ ]:
n_classes

{0.001: 1,
 0.003: 2,
 0.005: 5,
 0.007: 8,
 0.009000000000000001: 9,
 0.011: 8,
 0.013000000000000001: 8,
 0.015: 6,
 0.017: 6,
 0.019000000000000003: 4,
 0.021: 5,
 0.023: 4,
 0.025: 3,
 0.027000000000000003: 3,
 0.029: 3,
 0.031: 3,
 0.033: 3,
 0.035: 2,
 0.037000000000000005: 2,
 0.039: 2,
 0.041: 2,
 0.043000000000000003: 2,
 0.045: 2,
 0.047: 2,
 0.049: 2,
 0.051000000000000004: 2,
 0.053000000000000005: 2,
 0.055: 2,
 0.057: 2,
 0.059000000000000004: 2,
 0.061: 2,
 0.063: 2,
 0.065: 2,
 0.067: 2,
 0.069: 2,
 0.07100000000000001: 2,
 0.07300000000000001: 1,
 0.075: 1,
 0.077: 1,
 0.079: 1,
 0.081: 1,
 0.083: 1,
 0.085: 1,
 0.08700000000000001: 1,
 0.089: 1,
 0.091: 1,
 0.093: 1,
 0.095: 1,
 0.097: 1,
 0.099: 1,
 0.101: 1,
 0.10300000000000001: 1,
 0.10500000000000001: 1,
 0.107: 1,
 0.109: 1,
 0.111: 1,
 0.113: 1,
 0.115: 1,
 0.117: 1,
 0.11900000000000001: 1,
 0.121: 1,
 0.123: 1,
 0.125: 1,
 0.127: 1,
 0.129: 1,
 0.131: 1,
 0.133: 1,
 0.135: 1,
 0.137: 1,
 0.139: 1,
 0.14100000

In [ ]:
print( dbscan.labels_ )

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


In [ ]:
for cluster in range( max( dbscan.labels_ ) + 1 ):
  print( "CLUSTER " + str( cluster ) + " SENTENCES:" )
  for lbl in range( len( dbscan.labels_ ) ):
    if dbscan.labels_[lbl] == cluster:
      print( res.json()['data']['children'][lbl]['data']['title'] )
  print( "" )

CLUSTER 0 SENTENCES:
Drank last night
30days
Stomach pain in the night?
Tuesday Morning, EST 6:19 am / IWNDWYT
How do you utilize SD? I’m curious how to best utilize this sub.
Day 50….first doctors appointment in years
day one - here we go again!
On Day 4 now went to a friend yesterday to help renovate his house.
Non alcoholic alternatives
Ugh
I'm tapering at home with the guidance of a professional psychologist. 10+ years drinking is enough.
Today begins a Journey starting Early !
Day 3. I want to live my life.
Could there be a better timing? :D
day 3
Day 6 - This is pretty sick...
Day 10
I would have laughed at this comment not too long ago… but it may help somebody
Can’t sleep
Any advice?
I want to quit drinking and vaping, but I'm not strong enough to do both at once, and if I quit one, I get more heavy into the other.
187 Days and I'm Going to a Party This Weekend
So long as I follow that way I have nothing to fear. Just for today.
Lost my gf due to drinking every night. Found a s

In [ ]:
test_doc = word_tokenize("Had a passing thought in the shower about how nice it would be to get drunk on the beach this summer".lower())
test_doc_vector = model.infer_vector(test_doc)
model.docvecs.most_similar(positive = [test_doc_vector])

[(6, 0.9782257080078125),
 (63, 0.9751677513122559),
 (93, 0.970664381980896),
 (9, 0.9704173803329468),
 (40, 0.9696348905563354),
 (52, 0.9688957333564758),
 (51, 0.9688401222229004),
 (53, 0.9684365391731262),
 (44, 0.9662896394729614),
 (76, 0.9662660360336304)]

In [ ]:
for pair in model.docvecs.most_similar(positive = [test_doc_vector]):
  print( res.json()['data']['children'][pair[0]]['data']['title'] )

day one - here we go again!
Said goodbye today
5th Birthday Blues
Ugh
Making a step.
Second-day hangover...
Longest 9 days of my life
Time for a change.
24 hours… again
Losing Myself


### SentenceBERT

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 2.8 MB/s 
     |████████████████████████████████| 4.2 MB 7.9 MB/s 
     |████████████████████████████████| 1.2 MB 44.4 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 6.6 MB 24.8 MB/s 
     |████████████████████████████████| 596 kB 35.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0d4732652007ede3d010ac9c5d714efae01938e567f1ae5c0ec50d472eb41900
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
sentence_embeddings = sbert_model.encode(sentences)

In [ ]:
query = "My husband and I ordered a pizza and pasta and both were great, the waitress was really kind too"
query_vec = sbert_model.encode([query])[0]

In [ ]:
for sentence in sentences:
  sim = cosine(query_vec, sbert_model.encode([sentence])[0])
  print("Sentence = ", sentence, "; similarity = ", sim)

Sentence =  Drank last night ; similarity =  0.11463995
Sentence =  30days ; similarity =  -0.015665997
Sentence =  Stomach pain in the night? ; similarity =  0.0316402
Sentence =  Tuesday Morning, EST 6:19 am / IWNDWYT ; similarity =  0.11286379
Sentence =  How do you utilize SD? I’m curious how to best utilize this sub. ; similarity =  0.12369802
Sentence =  Day 50….first doctors appointment in years ; similarity =  0.235203
Sentence =  day one - here we go again! ; similarity =  0.02254771
Sentence =  On Day 4 now went to a friend yesterday to help renovate his house. ; similarity =  0.33247757
Sentence =  Non alcoholic alternatives ; similarity =  0.027988736
Sentence =  Ugh ; similarity =  -0.122177094
Sentence =  I'm tapering at home with the guidance of a professional psychologist. 10+ years drinking is enough. ; similarity =  0.43682882
Sentence =  Today begins a Journey starting Early ! ; similarity =  0.014327025
Sentence =  Day 3. I want to live my life. ; similarity =  0.13

In [ ]:
sent_vecs = {}

for i in range( len( sentence_embeddings ) ):
  sent_vecs.update( {df.title[i]: sentence_embeddings[i]} )

sentences = list( sent_vecs.keys() )
vectors = list( sent_vecs.values() )

In [ ]:
x = np.array( vectors )

n_classes = {}

for i in tqdm( np.arange( 0.001, 1, 0.002) ):
  dbscan = DBSCAN( eps=i, min_samples=2, metric='cosine' ).fit(x)
  n_classes.update( {i: len( pd.Series( dbscan.labels_ ).value_counts() ) } )

dbscan = DBSCAN( eps=0.20, min_samples=2, metric='cosine' ).fit(x) 

100%|██████████| 500/500 [00:02<00:00, 178.59it/s]


In [ ]:
n_classes

{0.001: 1,
 0.003: 1,
 0.005: 1,
 0.007: 1,
 0.009000000000000001: 1,
 0.011: 1,
 0.013000000000000001: 1,
 0.015: 1,
 0.017: 1,
 0.019000000000000003: 1,
 0.021: 1,
 0.023: 1,
 0.025: 1,
 0.027000000000000003: 3,
 0.029: 3,
 0.031: 3,
 0.033: 3,
 0.035: 3,
 0.037000000000000005: 3,
 0.039: 3,
 0.041: 3,
 0.043000000000000003: 3,
 0.045: 3,
 0.047: 3,
 0.049: 3,
 0.051000000000000004: 3,
 0.053000000000000005: 3,
 0.055: 3,
 0.057: 3,
 0.059000000000000004: 3,
 0.061: 3,
 0.063: 3,
 0.065: 3,
 0.067: 3,
 0.069: 3,
 0.07100000000000001: 3,
 0.07300000000000001: 3,
 0.075: 3,
 0.077: 3,
 0.079: 3,
 0.081: 3,
 0.083: 3,
 0.085: 3,
 0.08700000000000001: 3,
 0.089: 3,
 0.091: 3,
 0.093: 3,
 0.095: 3,
 0.097: 3,
 0.099: 3,
 0.101: 3,
 0.10300000000000001: 3,
 0.10500000000000001: 3,
 0.107: 3,
 0.109: 3,
 0.111: 3,
 0.113: 3,
 0.115: 3,
 0.117: 3,
 0.11900000000000001: 3,
 0.121: 3,
 0.123: 3,
 0.125: 3,
 0.127: 3,
 0.129: 3,
 0.131: 3,
 0.133: 3,
 0.135: 3,
 0.137: 3,
 0.139: 3,
 0.14100000

In [ ]:
print( dbscan.labels_ )

[-1  0 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1
 -1  3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4 -1 -1  5  2  1
 -1 -1  5 -1 -1 -1 -1 -1 -1 -1  6  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1
  3 -1  4 -1  5 -1 -1 -1  1 -1 -1  5  6 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1  0]


In [ ]:
for cluster in range( max( dbscan.labels_ ) + 1 ):
  print( "CLUSTER " + str( cluster ) + " SENTENCES:" )
  for lbl in range( len( dbscan.labels_ ) ):
    if dbscan.labels_[lbl] == cluster:
      print( res.json()['data']['children'][lbl]['data']['title'] )
  print( "" )

CLUSTER 0 SENTENCES:
30days
30 days.

CLUSTER 1 SENTENCES:
day one - here we go again!
Day 1 for me again
Day One
Day 1
Day 2

CLUSTER 2 SENTENCES:
I want to quit drinking and vaping, but I'm not strong enough to do both at once, and if I quit one, I get more heavy into the other.
Sleep problems makes me want to stop drinking

CLUSTER 3 SENTENCES:
Almost five days sober
G/F of 5 years left over my drinking

CLUSTER 4 SENTENCES:
My first sober date experience WOW
Going on holiday - first one sober. Any tips please?

CLUSTER 5 SENTENCES:
I hate drinking
This sucks.
Losing Myself
Having a hard time

CLUSTER 6 SENTENCES:
It’s Official
I’m done.



### Universal Sentence Encoder

In [ ]:
!pip3 install --upgrade tensorflow-gpu
!pip3 install tensorflow-hub

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.1 kB/s 
     |████████████████████████████████| 5.8 MB 67.5 MB/s 
     |████████████████████████████████| 438 kB 66.8 MB/s 
     |████████████████████████████████| 1.6 MB 55.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
sentence_embeddings = model(sentences)

In [ ]:
query = "We went for brunch and sat outside."
query_vec = model([query])[0]

In [ ]:
for sentence in sentences:
  sim = cosine(query_vec, model([sentence])[0])
  print("Sentence = ", sentence, "; similarity = ", sim)

Sentence =  Drank last night ; similarity =  0.16401835
Sentence =  30days ; similarity =  0.014903471
Sentence =  Stomach pain in the night? ; similarity =  0.022233555
Sentence =  Tuesday Morning, EST 6:19 am / IWNDWYT ; similarity =  0.19989231
Sentence =  How do you utilize SD? I’m curious how to best utilize this sub. ; similarity =  0.017074488
Sentence =  Day 50….first doctors appointment in years ; similarity =  0.093621984
Sentence =  day one - here we go again! ; similarity =  0.12794451
Sentence =  On Day 4 now went to a friend yesterday to help renovate his house. ; similarity =  0.12084878
Sentence =  Non alcoholic alternatives ; similarity =  -0.033420943
Sentence =  Ugh ; similarity =  0.043680146
Sentence =  I'm tapering at home with the guidance of a professional psychologist. 10+ years drinking is enough. ; similarity =  -0.037389833
Sentence =  Today begins a Journey starting Early ! ; similarity =  0.10662141
Sentence =  Day 3. I want to live my life. ; similarity =

In [ ]:
for sentence in sentences:
  sim = cosine(query_vec, model([sentence])[0])
  if sim > 0.2:
    print("Sentence = ", sentence, "; similarity = ", sim)

In [ ]:
sent_vecs = {}

for i in range( len( sentence_embeddings ) ):
  sent_vecs.update( {df.title[i]: sentence_embeddings[i]} )

sentences = list( sent_vecs.keys() )
vectors = list( sent_vecs.values() )

In [ ]:
x = np.array( vectors )

n_classes = {}

for i in tqdm( np.arange( 0.001, 1, 0.002) ):
  dbscan = DBSCAN( eps=i, min_samples=2, metric='cosine' ).fit(x)
  n_classes.update( {i: len( pd.Series( dbscan.labels_ ).value_counts() ) } )

dbscan = DBSCAN( eps=0.40, min_samples=2, metric='cosine' ).fit(x) 

100%|██████████| 500/500 [00:01<00:00, 443.40it/s]


In [ ]:
n_classes

{0.001: 1,
 0.003: 1,
 0.005: 1,
 0.007: 1,
 0.009000000000000001: 1,
 0.011: 1,
 0.013000000000000001: 1,
 0.015: 1,
 0.017: 1,
 0.019000000000000003: 1,
 0.021: 1,
 0.023: 1,
 0.025: 1,
 0.027000000000000003: 1,
 0.029: 1,
 0.031: 1,
 0.033: 1,
 0.035: 1,
 0.037000000000000005: 1,
 0.039: 1,
 0.041: 1,
 0.043000000000000003: 1,
 0.045: 1,
 0.047: 1,
 0.049: 1,
 0.051000000000000004: 1,
 0.053000000000000005: 1,
 0.055: 1,
 0.057: 1,
 0.059000000000000004: 1,
 0.061: 1,
 0.063: 1,
 0.065: 1,
 0.067: 1,
 0.069: 1,
 0.07100000000000001: 1,
 0.07300000000000001: 1,
 0.075: 1,
 0.077: 1,
 0.079: 1,
 0.081: 1,
 0.083: 1,
 0.085: 1,
 0.08700000000000001: 1,
 0.089: 1,
 0.091: 1,
 0.093: 1,
 0.095: 1,
 0.097: 1,
 0.099: 1,
 0.101: 1,
 0.10300000000000001: 1,
 0.10500000000000001: 1,
 0.107: 1,
 0.109: 1,
 0.111: 1,
 0.113: 1,
 0.115: 1,
 0.117: 1,
 0.11900000000000001: 2,
 0.121: 2,
 0.123: 2,
 0.125: 2,
 0.127: 2,
 0.129: 2,
 0.131: 2,
 0.133: 2,
 0.135: 2,
 0.137: 2,
 0.139: 2,
 0.14100000

In [ ]:
print( dbscan.labels_ )

[-1  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1 -1  1 -1 -1 -1 -1
 -1  2 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3  3  0
  0 -1 -1 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0
 -1 -1 -1 -1 -1  2 -1 -1  0 -1  3 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1
 -1 -1 -1  0]


In [ ]:
for cluster in range( max( dbscan.labels_ ) + 1 ):
  print( "CLUSTER " + str( cluster ) + " SENTENCES:" )
  for lbl in range( len( dbscan.labels_ ) ):
    if dbscan.labels_[lbl] == cluster:
      print( res.json()['data']['children'][lbl]['data']['title'] )
  print( "" )

CLUSTER 0 SENTENCES:
30days
day one - here we go again!
day 3
Day 10
Day 13 complete.
Day 1 for me again
52 days
Day 50.
Day One
Day 1
Day 2
30 days.

CLUSTER 1 SENTENCES:
Any advice?
Need Tips

CLUSTER 2 SENTENCES:
Almost five days sober
101 days sober

CLUSTER 3 SENTENCES:
I hate drinking
Sleep problems makes me want to stop drinking
I'm drinking too much. I don't know how to stop.



## Analyzing Paragraph Text

### Sentence-by-Sentence

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
sentences = []

for text in tqdm( df.selftext ):
  sentence_list = nltk.tokenize.sent_tokenize( text )
  for sentence in sentence_list:
    sentences.append( sentence )

100%|██████████| 100/100 [00:00<00:00, 1658.92it/s]


#### spaCy

In [ ]:
NLP = spacy.load( 'en_core_web_lg' )

sent_vecs = {}
docs = []

for sentence in tqdm( sentences ):
  doc = NLP( sentence )
  docs.append( doc )
  sent_vecs.update( {sentence: doc.vector} )

sentences = list( sent_vecs.keys() )
vectors = list( sent_vecs.values() ) 

NameError: ignored

In [ ]:
x = np.array( vectors )

n_classes = {}

for i in tqdm( np.arange( 0.001, 1, 0.002) ):
  dbscan = DBSCAN( eps=i, min_samples=2, metric='cosine' ).fit(x)
  n_classes.update( {i: len( pd.Series( dbscan.labels_ ).value_counts() ) } )

dbscan = DBSCAN( eps=0.099, min_samples=2, metric='cosine' ).fit(x) 

In [ ]:
n_classes

In [ ]:
print( dbscan.labels_ )

In [ ]:
for cluster in range( max( dbscan.labels_ ) + 1 ):
  print( "CLUSTER " + str( cluster ) + " SENTENCES:" )
  for lbl in range( len( dbscan.labels_ ) ):
    if dbscan.labels_[lbl] == cluster:
      print( sentences[lbl] )
  print( "" )

### By Full Post